In [192]:
# get the required table from wikipedia
import pandas as pd
import requests

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

wiki_df = pd.read_html(website_url,skiprows=0)[0]

wiki_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [193]:
# remove 'Not assigned' Boroughs
wiki_df = wiki_df[wiki_df['Borough'] != 'Not assigned']

# check the known issue (some Neighbourhoods could be 'Not assigned')
print(wiki_df[wiki_df['Neighbourhood'] == 'Not assigned'])


  Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Not assigned


In [194]:
#fix the rows that have 'Not assigned' Neighbourhoods so that the Neighbourhood shows the Borough for these cases
for i, row in wiki_df.iterrows():
    if (row[2] == "Not assigned"):
        row[2] = row[1]

# check to see if M7A is 'fixed'
print(wiki_df[wiki_df['Postcode'] == 'M7A'])

  Postcode       Borough Neighbourhood
8      M7A  Queen's Park  Queen's Park


In [195]:
# group the Neighbourhoods by Postcode and Borough
wiki_df = wiki_df.groupby(['Postcode','Borough']).Neighbourhood.unique().apply(lambda x: ', '.join(x)).reset_index()

# check to see if the grouping worked
wiki_df[wiki_df['Postcode'] == 'M5A']

,Postcode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [196]:
wiki_df.shape

# QUESTION 1: 103 records with three columns

(103, 3)

## Part 1: 103 records with three columns in cleaned data frame

In [197]:
# I could not get geocoder to work properly, so I will use the provided csv file

# read the csv file from my directory
coords = pd.read_csv("Geospatial_Coordinates.csv")

# rename the column in coords to match wiki_df
coords.rename(columns={'Postal Code':'Postcode'}, inplace=True)

# merge the dataframes to new 'my_df'
my_df = pd.merge(wiki_df, coords, on='Postcode')

# check validity of coordinates assigned with known value
my_df[my_df["Postcode"] == "M2H"]

,Postcode,Borough,Neighbourhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452


In [205]:
my_df.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Part 2 Completed here

In [206]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, Ontario, Canada'  # base of map

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
# get coordinated for Toronto city center and print the results
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [209]:
import folium # map rendering library

# create map of Toronto using latitude and longitude values
map_TO = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map from the neighbourhood data
for lat, lng, borough, neighborhood in zip(my_df['Latitude'], my_df['Longitude'], my_df['Borough'], my_df['Neighbourhood']):
    label = '{}; {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TO)  
    
map_TO

In [208]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

CLIENT_ID = '443E05IF1V3S2NAHVOA0CIDJGJQ1NNZZ2MH14NYFVXBGGG2K' # your Foursquare ID
CLIENT_SECRET = 'S13WDCLFB4JU44WNKX2ANWFXQGQGB0JXT4AOKZCW0AE0ZZ54' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [211]:
#get a subset of venues from Scarborough
scar_df = my_df[my_df['Borough'] == "Scarborough"]
scar_df.shape

# get the nearby venues for the TO neighboourhoods in the Scarborough Subset
toronto_venues = getNearbyVenues(names=scar_df['Neighbourhood'],
                                   latitudes=scar_df['Latitude'],
                                   longitudes=scar_df['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West
Upper Rouge


In [230]:
print(f"There are {toronto_venues.shape[0]} total venues in Scarborough and {toronto_venues['Venue Category'].nunique()} unique venue categories.")

There are 86 total venues in Scarborough and 52 unique venue categories.


In [231]:
toronto_venues.head(2)  # Check the data with Scarborough filter

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar


In [232]:
# 86 Scarborough venues with 52 unique categories

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [233]:
# group by the onehot Neighbourhood, averaging the results to create an weighted value for each Venue Category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,...,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.2,0.000000,0.000000,...,0.0,0.0,0.2,0.0,0.2,0.000000,0.0,0.000000,0.0,0.0
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.5,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.2,0.000000,0.0,0.000000,0.0,0.0
3,Cedarbrae,0.0,0.142857,0.0,0.142857,0.142857,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.142857,0.0,0.0
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.0,0.222222,0.000000,0.0,0.0,0.222222,0.111111,...,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.000000,0.0,0.0


In [234]:
toronto_grouped.shape
# check the shape of the data frame (expecting 16 boroughs in scarborough with 53 venue categories)

(16, 53)

In [237]:
# create a function that will sort the top venues for each neighborhood submitted in a data frame

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [274]:
import numpy as np
num_top_venues = 10  # get the 10 most popular venues for each neighbourhood

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Lounge,Sandwich Place,Breakfast Spot,Chinese Restaurant,Vietnamese Restaurant,College Stadium,General Entertainment,Fried Chicken Joint,Fast Food Restaurant
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Park,Playground,Chinese Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Discount Store
2,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Farm,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
3,Cedarbrae,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store,General Entertainment
4,"Clairlea, Golden Mile, Oakridge",Bakery,Bus Line,Intersection,Soccer Field,Fast Food Restaurant,Bus Station,Metro Station,Convenience Store,Grocery Store,General Entertainment


In [275]:
# set number of clusters I experiemented with 3 through 10 selecting 7 as the best number of clusters (most differentiation)
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 6, 3, 3, 3, 3, 5, 3, 3, 3], dtype=int32)

In [276]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = scar_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the dataframe

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Vietnamese Restaurant,Thrift / Vintage Store,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Farm,Electronics Store,Discount Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,2.0,Bar,Vietnamese Restaurant,College Stadium,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,3.0,Intersection,Electronics Store,Breakfast Spot,Rental Car Location,Medical Center,Pizza Place,Mexican Restaurant,College Stadium,General Entertainment,Fried Chicken Joint
3,M1G,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean Restaurant,Indian Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Farm,Electronics Store,Discount Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,3.0,Hakka Restaurant,Thai Restaurant,Athletics & Sports,Bakery,Bank,Fried Chicken Joint,Caribbean Restaurant,Convenience Store,Grocery Store,General Entertainment


In [277]:
#toronto_merged = toronto_merged[[toronto_merged['Cluster Labels'] != 'nan']]
toronto_merged['Cluster Labels'].replace([np.inf, -np.inf], np.nan)
toronto_merged['Cluster Labels'].fillna(0, inplace=True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged['Cluster Labels'].value_counts()
# check to see how many of each cluster

3    9
6    2
0    2
5    1
4    1
2    1
1    1
Name: Cluster Labels, dtype: int64

In [283]:
address = 'Scarborough, Ontario, Canada'  # base of map

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
# get coordinated for Scarborough city center and print the results
latitude = location.latitude
longitude = location.longitude

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create the scarborough map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set colors for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color= rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters.save("scarb.jpg")
map_clusters

##### Noticings: 
### The Boroughs on the west side (closer to downtown) are very similar neighbourhoods.  The Boroughs become more differentiated the further you move away from the city core.
<img src="scarb.png">